In [1]:
using Pkg
Pkg.activate("EnvFerrite")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


In [2]:
using TypedPolynomials
@polyvar x y

(x, y)

In [3]:
p = 2x*y^2 +y -2x
q = 3x*y^3 - 4x + 4

4 - 4x + 3xy³

In [4]:
function nabladotnabla(a,b)
    diff_a = differentiate(a,(x,y))
    diff_b = differentiate(b,(x,y))
    return diff_a[1]*diff_b[1]+diff_a[2]*diff_b[2]
end

nabladotnabla (generic function with 1 method)

In [5]:
r = nabladotnabla(p,q)

8 - 8y² - 6y³ + 9xy² + 6y⁵ + 36x²y³

In [6]:
using Ferrite
using SparseArrays
using LinearAlgebra
using IterativeSolvers


In [7]:
grid = generate_grid(Quadrilateral, (16, 16));
dim = Ferrite.getspatialdim(grid)
order = 3
qr_order = 5
ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(qr_order)
cellvalues = CellValues(qr, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)

DofHandler{2, Grid{2, Quadrilateral, Float64}}
  Fields:
    :u, Lagrange{RefQuadrilateral, 3}()
  Dofs per cell: 16
  Total dofs: 2401

In [8]:
# for that we also gonna assemble the Mass Matrix:
# Mass Matrix  ∫(u*v)dΩ
function assemble_M(cellvalues::CellValues,dh::DofHandler)
    M = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Me = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Me, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)      
            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    φⱼ = shape_value(cellvalues, q_point, j)
                    Me[i,j] += φᵢ * φⱼ * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Me)
    end
    return M, cholesky(M)
end   
M,MC = assemble_M(cellvalues,dh)

(sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  2386, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2401], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2401, 2401, 2401, 2401, 2401, 2401, 2401, 2401, 2401, 2401], [9.070294784580477e-5, 1.346371882086135e-5, 1.9985207624718373e-6, 1.346371882086154e-5, 7.01530612244898e-5, -2.5510204081632996e-5, 1.0413345025508904e-5, -3.7866709183683842e-6, 1.0413345025508914e-5, -3.786670918368958e-6  …  -0.00012914540816327312, 5.4259008290814714e-5, -4.439373405613032e-5, 0.0003551498724489819, -4.439373405612036e-5, 0.0003551498724489678, 3.6322146045981375e-5, -0.0002905771683673128, -0.00029057716836730684, 0.0023246173469387552], 2401, 2401), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  58833
nnz:     58833
success: true
)

In [9]:
# now we gonna project conductivity unto a vector:
function assemble_function_vector(cellvalues::CellValues, dh::DofHandler, f, M)
    F = zeros(ndofs(dh))
    n_basefuncs = getnbasefunctions(cellvalues)
    Fe = zeros(n_basefuncs)
    cdofs = zeros(Int, n_basefuncs)

    for cell in CellIterator(dh)
        fill!(Fe, 0.0)
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        cdofs = celldofs(cell)
        for q in 1:getnquadpoints(cellvalues)
            x_q = spatial_coordinate(cellvalues, q, coords)
            f_val = f(x_q)
            dΩ = getdetJdV(cellvalues, q)

            for i in 1:n_basefuncs
                Fe[i] += f_val * shape_value(cellvalues, q, i) * dΩ
            end
        end  
        assemble!(F, cdofs,Fe)
    end
    return M \ F

end

assemble_function_vector (generic function with 1 method)

In [10]:
function get_func(p)
    (z) -> p(x=>z[1],y=>z[2])
end

get_func (generic function with 1 method)

In [11]:
p_func = get_func(p)
q_func = get_func(q)
r_func = get_func(r)

#11 (generic function with 1 method)

In [12]:
p_vec = assemble_function_vector(cellvalues, dh, p_func, MC)
q_vec = assemble_function_vector(cellvalues, dh, q_func, MC)
r_vec = assemble_function_vector(cellvalues, dh, r_func, MC)

2401-element Vector{Float64}:
 -44.99989987093435
 -35.43739987093471
 -21.676848793324638
 -28.190642738637912
 -41.687399870935145
 -38.49989987093439
 -30.407491490219304
 -25.831390470881562
 -25.93563785582463
 -23.76437320738624
   ⋮
  44.99989987093272
  35.831281963937236
  40.172397631711746
  38.49989987093387
  41.687399870932865
  30.772109510233868
  33.25355482273406
  34.4228135749995
  37.242597103644314

now we can write an assembler for $\nabla(a)\cdot\nabla(b)$

In [13]:
function calculate_bilinear_map_rhs(a::AbstractVector, b::AbstractVector, cellvalues::CellValues, dh::DofHandler, M)
    n = ndofs(dh)
    rhs = zeros(n)
    n_basefuncs = getnbasefunctions(cellvalues)
    qpoints = getnquadpoints(cellvalues)
    re = zeros(n_basefuncs)
    
    for cell in CellIterator(dh)
        dofs = celldofs(cell)
        reinit!(cellvalues, cell)
        fill!(re, 0.0)
        
        # Extract local DOF values for this cell
        ae = a[dofs]
        be = b[dofs]
        
        for q in 1:qpoints
            dΩ = getdetJdV(cellvalues, q)
            
            # Compute gradients of a and b at this quadrature point
            ∇a_q = zero(Vec{2,Float64})  # assuming 2D
            ∇b_q = zero(Vec{2,Float64})
            
            for j in 1:n_basefuncs
                ∇ϕⱼ = shape_gradient(cellvalues, q, j)
                ∇a_q += ae[j] * ∇ϕⱼ
                ∇b_q += be[j] * ∇ϕⱼ
            end
            
            # Compute ∇a⋅∇b at this quadrature point
            grad_dot_product = ∇a_q ⋅ ∇b_q
            
            # Integrate against test functions
            for i in 1:n_basefuncs
                ϕᵢ = shape_value(cellvalues, q, i)
                re[i] += grad_dot_product * ϕᵢ * dΩ
            end
        end
        
        assemble!(rhs, dofs, re)
    end
    
    return M \ rhs
end

calculate_bilinear_map_rhs (generic function with 1 method)

In [14]:
r_test = calculate_bilinear_map_rhs(p_vec,q_vec, cellvalues,dh,MC)

2401-element Vector{Float64}:
 -44.9998998708417
 -35.437399870909296
 -21.676848793286936
 -28.190642738607878
 -41.68739987098482
 -38.499899870848886
 -30.40749149022063
 -25.83139047087344
 -25.93563785581886
 -23.764373207270793
   ⋮
  44.99989987091296
  35.83128196394736
  40.172397631711135
  38.49989987091419
  41.68739987093987
  30.772109510236298
  33.2535548227315
  34.422813574998315
  37.24259710363415

In [15]:
@assert norm(r_vec - r_test) < 100.0

In [16]:
norm(r_vec - r_test)

6.050063388918774e-10

In [17]:

# Test function: a smooth function with known analytical TV gradient
# u(x,y) = sin(πx) * cos(πy)
# This has a well-defined gradient everywhere and we can compute TV analytically
function create_test_function()
    u_analytical(coords) = sin(π * coords[1]) * cos(π * coords[2])
    
    # Analytical gradient: ∇u = [π*cos(πx)*cos(πy), -π*sin(πx)*sin(πy)]
    function grad_u_analytical(coords)
        x, y = coords[1], coords[2]
        return [π * cos(π * x) * cos(π * y), -π * sin(π * x) * sin(π * y)]
    end
    
    # Analytical TV gradient: ∇·(∇u/|∇u|)
    # This is complex, so we'll use finite differences for verification
    function tv_gradient_analytical_approx(coords)
        h = 1e-6
        x, y = coords[1], coords[2]
        
        # Compute |∇u| at neighboring points
        grad_center = grad_u_analytical(coords)
        grad_norm_center = norm(grad_center)
        
        if grad_norm_center < 1e-12
            return 0.0  # Handle singularity
        end
        
        # Finite difference approximation of ∇·(∇u/|∇u|)
        # ∂/∂x (∇u_x/|∇u|) + ∂/∂y (∇u_y/|∇u|)
        
        grad_xplus = grad_u_analytical([x + h, y])
        grad_xminus = grad_u_analytical([x - h, y])
        grad_yplus = grad_u_analytical([x, y + h])
        grad_yminus = grad_u_analytical([x, y - h])
        
        norm_xplus = norm(grad_xplus)
        norm_xminus = norm(grad_xminus)
        norm_yplus = norm(grad_yplus)
        norm_yminus = norm(grad_yminus)
        
        # Avoid division by zero
        ux_norm_xplus = norm_xplus > 1e-12 ? grad_xplus[1] / norm_xplus : 0.0
        ux_norm_xminus = norm_xminus > 1e-12 ? grad_xminus[1] / norm_xminus : 0.0
        uy_norm_yplus = norm_yplus > 1e-12 ? grad_yplus[2] / norm_yplus : 0.0
        uy_norm_yminus = norm_yminus > 1e-12 ? grad_yminus[2] / norm_yminus : 0.0
        
        # Central differences
        d_dx_term = (ux_norm_xplus - ux_norm_xminus) / (2 * h)
        d_dy_term = (uy_norm_yplus - uy_norm_yminus) / (2 * h)
        
        return d_dx_term + d_dy_term
    end
    
    return u_analytical, grad_u_analytical, tv_gradient_analytical_approx
end


create_test_function (generic function with 1 method)

In [22]:
u_ana, gra_u, tv_grad = create_test_function()

(var"#u_analytical#13"(), var"#grad_u_analytical#14"(), var"#tv_gradient_analytical_approx#15"{var"#grad_u_analytical#14"}(var"#grad_u_analytical#14"()))

In [23]:
tv_func = assemble_function_vector(cellvalues, dh, tv_grad, MC)

2401-element Vector{Float64}:
  0.02271021763941463
 -1.180782982127064
 -1.7140549430105632
 -0.2090920234889143
 -0.39630223038378665
 -0.860914084303656
 -1.3544649261037012
 -1.4755395690318578
 -0.5870676614894398
 -1.1165422878986235
  ⋮
 -0.022710217609266302
 -0.03291605212047728
  0.012844310463726848
  0.860914084314232
  0.3963022303536433
  0.9601363660359022
  0.4673812301791181
  0.8685580773715618
  0.4310663310246094

In [18]:

# TV Regularization assembler
function assemble_tv_gradient(u_vec::AbstractVector, cellvalues::CellValues, dh::DofHandler, M, ε::Float64 = 1e-8)
    """
    Assembles the TV regularization term: ∇·(∇u/√(|∇u|² + ε²))
    
    Args:
        u_vec: DOF vector of the function u
        cellvalues: Finite element cell values
        dh: DOF handler
        M: Mass matrix factorization for solving
        ε: Regularization parameter to avoid division by zero
    
    Returns:
        tv_grad_vec: L² projection of the TV gradient
        error_estimate: Error estimate (based on residual norm)
    """
    
    n = ndofs(dh)
    rhs = zeros(n)
    n_basefuncs = getnbasefunctions(cellvalues)
    qpoints = getnquadpoints(cellvalues)
    re = zeros(n_basefuncs)
    
    total_residual = 0.0
    total_volume = 0.0
    
    for cell in CellIterator(dh)
        dofs = celldofs(cell)
        reinit!(cellvalues, cell)
        fill!(re, 0.0)
        
        # Extract local DOF values for this cell
        ue = u_vec[dofs]
        
        for q in 1:qpoints
            dΩ = getdetJdV(cellvalues, q)
            total_volume += dΩ
            
            # Compute gradient of u at this quadrature point
            ∇u_q = zero(Vec{2,Float64})  # assuming 2D
            
            for j in 1:n_basefuncs
                ∇ϕⱼ = shape_gradient(cellvalues, q, j)
                ∇u_q += ue[j] * ∇ϕⱼ
            end
            
            # Compute |∇u|² + ε²
            grad_norm_sq = ∇u_q ⋅ ∇u_q + ε^2
            grad_norm = sqrt(grad_norm_sq)
            
            # Normalized gradient: ∇u/|∇u|_ε
            ∇u_normalized = ∇u_q / grad_norm
            
            # For TV gradient ∇·(∇u/|∇u|_ε), we use integration by parts:
            # ∫ ∇·(∇u/|∇u|_ε) φᵢ dΩ = -∫ (∇u/|∇u|_ε)·∇φᵢ dΩ + boundary terms
            # (assuming zero boundary conditions, boundary terms vanish)
            
            for i in 1:n_basefuncs
                ∇ϕᵢ = shape_gradient(cellvalues, q, i)
                # Integration by parts: -∇u_normalized ⋅ ∇φᵢ
                re[i] += -∇u_normalized ⋅ ∇ϕᵢ * dΩ
            end
            
            # Estimate error (based on gradient magnitude variation)
            total_residual += grad_norm * dΩ
        end
        
        assemble!(rhs, dofs, re)
    end
    
    # Solve for TV gradient in L² sense
    tv_grad_vec = M \ rhs
    
    # Simple error estimate based on average gradient magnitude
    error_estimate = total_residual / total_volume
    
    return tv_grad_vec, error_estimate
end

assemble_tv_gradient (generic function with 2 methods)

In [21]:
vetv, tverror = assemble_tv_gradient(p_vec, cellvalues, dh, MC)
vetv

2401-element Vector{Float64}:
  123.93543137629521
  123.93546853761069
  -14.31706718531205
  -28.751295397722
  123.9354700648479
  123.93546241790786
  -18.47961170104244
    7.861494413715924
  -13.959303185654678
  -16.651560059381296
    ⋮
 -123.93543137632162
    1.6523445510256682
   23.18046909432949
 -123.93546241790911
 -123.9354700648355
   -6.3229276314927185
   -8.09771889626097
   19.239782521438855
   18.362753626173085

In [19]:

# Test function to verify the implementation
function test_tv_regularization(cellvalues, dh, M)
    """
    Test the TV regularization implementation
    """
    println("Testing TV Regularization...")
    
    # Create test function
    u_analytical, grad_u_analytical, tv_grad_analytical = create_test_function()
    
    # Project test function onto finite element space
    u_vec = assemble_function_vector(cellvalues, dh, u_analytical, M)
    
    # Compute TV gradient using our assembler
    tv_grad_vec, error_est = assemble_tv_gradient(u_vec, cellvalues, dh, M, 1e-6)
    
    # Project analytical TV gradient for comparison
    tv_grad_analytical_vec = assemble_function_vector(cellvalues, dh, tv_grad_analytical, M)
    
    # Compute L² error
    error_vec = tv_grad_vec - tv_grad_analytical_vec
    l2_error = sqrt(error_vec' * M * error_vec)
    
    println("L² error in TV gradient: ", l2_error)
    println("Error estimate: ", error_est)
    
    # For a smooth function like sin(πx)cos(πy), the error should be reasonable
    # but not too small since TV is designed for non-smooth functions
    success = l2_error < 1.0  # Relaxed tolerance since TV is approximate for smooth functions
    
    println("Test ", success ? "PASSED" : "FAILED")
    println("Expected: Error should be moderate (TV is for non-smooth functions)")
    
    return tv_grad_vec, l2_error, success
end


test_tv_regularization (generic function with 1 method)

In [20]:

# Additional test with a piecewise function (more suitable for TV)
function test_tv_piecewise(cellvalues, dh, M)
    """
    Test TV with a piecewise function (more natural for TV regularization)
    """
    println("\nTesting TV with piecewise function...")
    
    # Piecewise constant function with discontinuity
    function u_piecewise(coords)
        x, y = coords[1], coords[2]
        if x < 0.5
            return 1.0
        else
            return -1.0
        end
    end
    
    # Project onto FE space
    u_vec = assemble_function_vector(cellvalues, dh, u_piecewise, M)
    
    # Compute TV gradient
    tv_grad_vec, error_est = assemble_tv_gradient(u_vec, cellvalues, dh, M, 1e-6)
    
    println("TV gradient norm: ", norm(tv_grad_vec))
    println("Error estimate: ", error_est)
    println("Max TV gradient value: ", maximum(abs.(tv_grad_vec)))
    
    # For a piecewise function, TV gradient should be significant near discontinuities
    success = norm(tv_grad_vec) > 0.1  # Should have significant TV gradient
    
    println("Test ", success ? "PASSED" : "FAILED")
    println("Expected: Significant TV gradient due to discontinuity")
    
    return tv_grad_vec, success
end


test_tv_piecewise (generic function with 1 method)

In [21]:

# Simple test runner - add this to your existing code after the assert
function run_tv_tests()
    println("="^50)
    println("Running TV Regularization Tests")
    println("="^50)
    
    # Test 1: Smooth function
    tv_grad_smooth, l2_error_smooth, success1 = test_tv_regularization(cellvalues, dh, MC)
    
    # Test 2: Piecewise function  
    tv_grad_piecewise, success2 = test_tv_piecewise(cellvalues, dh, MC)
    
    println("\n" * "="^50)
    println("TV Tests Summary:")
    println("Smooth function test: ", success1 ? "PASSED" : "FAILED")
    println("Piecewise function test: ", success2 ? "PASSED" : "FAILED")
    println("="^50)
    
    return success1 && success2
end


run_tv_tests (generic function with 1 method)

In [23]:

# Uncomment the line below to run tests:
 run_tv_tests()

Running TV Regularization Tests
Testing TV Regularization...


MethodError: MethodError: no method matching *(::Adjoint{Float64, Vector{Float64}}, ::SparseArrays.CHOLMOD.Factor{Float64, Int64})
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  *(!Matched::ChainRulesCore.NoTangent, ::Any)
   @ ChainRulesCore ~/.julia/packages/ChainRulesCore/Vsbj9/src/tangent_arithmetic.jl:64
  *(::Any, !Matched::ChainRulesCore.ZeroTangent)
   @ ChainRulesCore ~/.julia/packages/ChainRulesCore/Vsbj9/src/tangent_arithmetic.jl:105
  ...
